In [1]:
import requests
import os

foursquare_api_key = os.getenv('Foursquare_api_key')
yelp_api_key = os.getenv('Yelp_api_key')

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
foursquare_url = "https://api.foursquare.com/v3/places/search?radius=1000"

foursquare_headers = {
    "accept": "application/json",
    "Authorization": f"{foursquare_api_key}"
}

response = requests.get(foursquare_url, headers=foursquare_headers)

# check the request format and whether response is success or not
if response.status_code == 200:
    response_json = response.json()
    print("Response is success")

else:
    print(f"Error: {response.status_code}, {response.text}")

Response is success


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [3]:
import pandas as pd

params = {
    'radius': 1000,
    'll': '40.7243,-74.0018',
    'query': 'coffee',
}
foursquare_response = requests.get(foursquare_url, headers=foursquare_headers)
df_foursquare_response = pd.DataFrame(response)
print(df_foursquare_response)

# saving as a csv file and it's easy to explore the data
df_foursquare_response.to_csv("C:/Users/kasun/Desktop/Data Analytics/Project 02/practice test/foursqaure.csv")

                                                    0
0   b'{"results":[{"fsq_id":"57a24949498ed4f80f6eb...
1   b'tor","plural_name":"Fence Contractors","icon...
2   b'}}],"chains":[],"closed_bucket":"LikelyOpen"...
3   b'nk":"/v3/places/57a24949498ed4f80f6eb780","l...
4   b'_address":"695 Secord Blvd NW, Edmonton AB T...
..                                                ...
62  b'Edmonton AB T5T 4K5","locality":"Edmonton","...
63  b'places":{"parent":{"fsq_id":"57d6e90bcd10062...
64  b'Plaza","plural_name":"Shopping Plazas","icon...
65  b'"name":"West Henday Promenade"}},"timezone":...
66       b',"longitude":-113.6579},"radius":1000}}}}'

[67 rows x 1 columns]


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [4]:
yelp_url = "https://api.yelp.com/v3/businesses/search"
yelp_params = {
    'location' : 'NYC'
}

yelp_headers = {
    "accept": "application/json",
    "Authorization": f"{yelp_api_key}"
}

response = requests.get(yelp_url, headers=yelp_headers, params=yelp_params)

# check the request format and whether response is success or not
if response.status_code == 200:
    response_json = response.json()
    print("Response is success")

else:
    print(f"Error: {response.status_code}, {response.text}")

Response is success


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [ ]:
yelp_params = {
    'radius' : 1000,
    'latitude': 40.7243,
    'longitude' : -74.0018,
    'query': 'coffee',
}

response_yelp = requests.get(yelp_url, headers=yelp_headers, params=yelp_params)

print(response_yelp.text)

In [6]:
data_yelp = response_yelp.json()
    
yelp_places = data_yelp.get('businesses', [])
df_yelp_places = pd.DataFrame(yelp_places)

df_yelp_places.to_csv("C:/Users/kasun/Desktop/Data Analytics/Project 02/practice test/yelp_data.csv")

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp API is likely to provide more complete details, especially if you need information like business ratings, reviews, and detailed addresses. This makes it more suitable for applications where comprehensive business data is essential.

Foursquare API, while potentially returning more results in terms of quantity (depending on the query and location), might not include as much detailed business information, making it less "complete" in that context. However, it could be more useful if you are specifically interested in location data or broader search categories.

Get the top 10 restaurants according to their rating

In [10]:
search_response = requests.get(url = foursquare_url, headers=foursquare_headers)
df_search_response = pd.DataFrame(search_response)
# print(df_search_response)
df_search_response.to_csv("C:/Users/kasun/Desktop/Data Analytics/Project 02/practice test/top 10 restaurant/foursqaure_restau_response.csv")

if search_response.status_code == 200:
    search_data = search_response.json()
    places = search_data.get('results', [])

    df_places = pd.DataFrame(places)
    df_places.to_csv("C:/Users/kasun/Desktop/Data Analytics/Project 02/practice test/top 10 restaurant/filtered_foursqaure_restau_response.csv")

else:
    print(f"Error: {search_response.status_code}, {search_response.text}")
    places = []

detail_url = 'https://api.foursquare.com/v3/places/'
detailed_places = []

for place in places:
    place_id = place['fsq_id']
    response = requests.get(f"{detail_url}{place_id}", headers=foursquare_headers)
    
    if response.status_code == 200:
        detailed_info = response.json()
        place['rating'] = detailed_info.get('rating', 0)
        detailed_places.append(place)
    else:
        print(f"Failed to get details for place ID {place_id}: {response.status_code}")

top_places = sorted(detailed_places, key=lambda x: x.get('rating', 1), reverse=True)[:10]

for i, place in enumerate(top_places, start=1):
    name = place.get('name', 'N/A')
    address = place.get('location', {}).get('formatted_address', 'N/A')
    rating = place.get('rating', 'N/A')
    
    print(f"{i}. {name} - {address} - Rating: {rating}")

1. Peter Cannon Construction - 695 Secord Blvd NW, Edmonton AB T5T 6Z6 - Rating: 0
2. Grian of Rice - 1312 Webber Greens Dr NW, Edmonton AB T5T 7C5 - Rating: 0
3. Tlc Independent Consultant - 8608 190A St NW, Edmonton AB T5T 4Y3 - Rating: 0
4. Park West - 189 St (87 Ave. to 95 Ave.), Edmonton AB - Rating: 0
5. Christian Reformed Churches - 19010 87 Ave NW, Edmonton AB T5T 6P2 - Rating: 0
6. Prestige Fairway Target Greens Ltd - 19101 87 Ave NW, Edmonton AB T5T 5Z7 - Rating: 0
7. Parkwest Pet Clinic - 18924 87 Ave NW, Edmonton AB T5T 6J1 - Rating: 0
8. Mac's Convenience Store - 18904 87 Ave NW (Few blocks from WEMl on 87th Ave Westbound), Edmonton AB T5T 6J1 - Rating: 0
9. 87 Pizza & Donair - 18914 87 Ave NW (87 Avenue), Edmonton AB T5T 6J1 - Rating: 0
10. Browns Socialhouse West Henday - 1010 Webber Greens Dr NW, Edmonton AB T5T 4K5 - Rating: 0
